# Ноутбук для проекта

https://github.com/MaryIzo/taro_bot/tree/master

Описание проекта:   
Часто бывает такое, что нужно принять какое-то важдое решение и "посоветоваться" с судьбой. Основываясь вероятностях и случайности и возможности человека обратиться с каким-то вопросом к искуственному интеллекту.  
Последнее время в интернете стали очень популярны гадания на картах таро, которые окружают себя ореолом таинственности, но по сути основаны на случайном выпадении карт в ответ на ваши вопросы.   
Давайте попробуем повторить это.  
    
План выполнения:   
1.  Добавим в модель промт, который превращает ее в гадалку
2.  Разобьем все возможные гадания на несколько вариантов: карта на день, прошлое настоящее будущее, вопрос, кельтский крест. От типа расклада зависит количество карт.    
3.  Используем RAG и случайность для того чтобы возвращать ответ на вопрос.  
4.  Надо разбить текст на части в которых есть:
     - описание каждого типа расклада
     - описание всех возможных карт
     - картинки всех возможных карт (можно попробовать использовать мультимодальную модель)


<center> <img src='https://github.com/a-milenkin/LLM_practical_course/blob/main/images/RAG.png?raw=1' width="800" height="250">


### Импорты всех необходимых библиотек

In [1]:
!python --version

Python 3.11.11


In [2]:
!pip install openai langchain tiktoken langchain-openai langchain-community sentence_transformers faiss-cpu pypdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [3]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.1 MB/s eta 0:00:0

In [4]:
!pip install transformers requests

In [5]:
from typing import List
from pypdf import PdfReader
import pandas as pd
from tqdm import tqdm
import numpy as np
import requests
import chromadb
import random
import pickle

In [8]:
import pypdf

In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

from langchain.vectorstores import FAISS
from google.colab import drive

## Предобработка текстов

### Собираем тексты для векторной базы

In [10]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [11]:
# creating a pdf reader object
reader = PdfReader('/content/drive/MyDrive/taro_bot/The Ultimate Guide to Tarot - A Beginner.pdf')
number_of_pages = len(reader.pages)

# printing number of pages in pdf file
print('Number of pages:', number_of_pages)

Number of pages: 493


In [12]:
docs = []
for j in range(41):
  # getting a specific page from the pdf file
  page = reader.pages[j]
  # extracting text from page
  text = page.extract_text()
  docs.append(text.replace('\t', ' ').replace('\n', ' '))

In [13]:
print(len(docs))

41


In [14]:
str_docs = '\n'.join(docs)

In [15]:
x = [len(one_a) for one_a in str_docs.split('\n')]
print('Средняя длина абзаца', np.mean(x))

Средняя длина абзаца 820.6341463414634


In [16]:
print(len(str_docs))

33686


### Сформируем тексты и картинки для всех карт

In [17]:
card_images = [
    45, 53, 62, 69, 78, 87, 96, 104, 113, 121, 129, 138, 147, 156, 164,
    173, 182, 191, 199, 208, 217, 226, 239, 243, 247, 251, 255, 259, 263, 266,
    267, 271, 275, 279, 283, 287, 291, 295, 299, 303, 307, 311, 315, 319, 323,
    327, 331, 335, 339, 343, 347, 351, 355, 359, 363, 367, 371,
    375, 379, 383, 387, 391, 395, 399, 403, 407, 411, 415, 419, 423,
    427, 431, 435, 439, 443, 447, 451, 455, 459
    ]
end_idx = 472
corrupted_pages = set([166])

In [19]:
def save_page_image(page, name):
  for count, image_file_object in enumerate(page.images):
    with open(f"""/content/drive/MyDrive/taro_bot/card_images/{name}""" + "__" + f"""{str(count) + image_file_object.name}""", "wb") as fp:
        fp.write(image_file_object.data)
    return name + "__" + f"""{str(count) + image_file_object.name}"""

def get_image_name(text):
  return text.split('\n')[0].lower().replace(' ', '_').replace(',', '').replace('—', '')

def get_card_description(short_description, img_name_idx, next_img_idx):
    card_description = [short_description]
    for k in range(img_name_idx+1, next_img_idx-1):
        if k in corrupted_pages:
            continue
        page = reader.pages[k]
        text = page.extract_text()
        card_description.append(text.replace('\t', ' '))
    return card_description

### Сохраняем все картинки в папочку card_images

In [20]:
cards_dictionary = {}

for j in range(len(card_images)):
  img = card_images[j]
  img_idx = img - 1
  img_name_idx = img - 2
  if j < len(card_images)-1:
    next_img_idx = card_images[j+1] - 1
  else:
    next_img_idx = end_idx - 1

  page = reader.pages[img_name_idx]
  short_description = page.extract_text().replace('\t', ' ')
  name = get_image_name(short_description)
  if name == 'number_of_earthand_so_the_potential_to_bring_heaven_down_to_earth_to_manifest':
    name = 'number_of_earthand'

  card_description = get_card_description(short_description, img_name_idx, next_img_idx)
  final_name = save_page_image(reader.pages[img_idx], name)

  cards_dictionary[final_name] = card_description

In [21]:
# print('\n'.join(card_description)

In [22]:
cnt = 0
cards_meaning_splits = {}

for key, val in cards_dictionary.items():
  # print('Description len: ', len(val))

  card_description = ' '.join(val)
  if card_description.find('UPRIGHT MEANING') != -1:
    upight_meaning = card_description[
        card_description.find('UPRIGHT MEANING'):card_description.find('REVERSED MEANING')
        ]
  else:
    upight_meaning = card_description[
        card_description.find('UNDERSTANDING'):
        ]
  # print(upight_meaning)

  if (
      upight_meaning.find('Home') != -1
      and upight_meaning.find('Relationships') != -1
      and upight_meaning.find('Career and money') != -1
      ):
    meaning_split = [upight_meaning[:upight_meaning.find('Home')].replace('\n', ' '),
        upight_meaning[upight_meaning.find('Home'):upight_meaning.find('Relationships')].replace('\n', ' '),
        upight_meaning[upight_meaning.find('Relationships'):upight_meaning.find('Career and money')].replace('\n', ' '),
        upight_meaning[upight_meaning.find('Career and money'):].replace('\n', ' ')
    ]
  else:
    meaning_split = upight_meaning.replace('\n', ' ').split('.')

  cards_meaning_splits[key] = meaning_split

  cnt += 1
  # for split in meaning_split:
  #   print(cnt, split)
  # print()

In [23]:
cards_splitted = list(cards_meaning_splits.values())

In [25]:
with open('/content/drive/MyDrive/taro_bot/cards_meaning_splits.pkl', 'wb') as f:
    pickle.dump(cards_meaning_splits, f)

### Создание коллекции для одной карты
Решила, что проще делать суммаризацию.

In [26]:
delete_mode = False
cards_collections = {}
chroma_client = chromadb.Client()

for key, val in cards_dictionary.items():
    print(key)
    if delete_mode:
        chroma_client.delete_collection(name=key)
    else:
        collection = chroma_client.get_or_create_collection(name=key)

        meaning_split = cards_meaning_splits[key]
        collection.upsert(
            documents=meaning_split,
            ids=[f'id{i}' for i in range(len(meaning_split))]
        )

0_the_fool__0Image0.jpg


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 77.2MiB/s]


i_the_magician__0Image0.jpg
ii_the_high_priestess__0Image0.jpg
iii_the_empress__0Image0.jpg
iv_the_emperor__0Image0.jpg
v_the_hierophant__0Image0.jpg
vi_the_lovers__0Image0.jpg
vii_the_chariot__0Image0.jpg
viii_strength__0Image0.jpg
ix_the_hermit__0Image0.jpg
x_the_wheel_of_fortune__0Image0.jpg
xi_justice__0Image0.jpg
xii_the_hanged_man__0Image0.jpg
xiii_death__0Image0.jpg
xiv_temperance__0Image0.jpg
xv_the_devil__0Image0.jpg
xvi_the_tower__0Image0.jpg
xvii_the_star__0Image0.jpg
xviii_the_moon__0Image0.jpg
xix_the_sun__0Image0.jpg
xx_judgment__0Image0.jpg
xxi_the_world__0Image0.jpg
the_suit_of_cups__0Image0.jpg
two_of_cups__0Image0.jpg
three_of_cups__0Image0.jpg
four_of_cups__0Image0.jpg
five_of_cups__0Image0.jpg
six_of_cups__0Image0.jpg
seven_of_cups__0Image0.jpg
number_of_earthand__0Image0.jpg
eight_of_cups__0Image0.jpg
nine_of_cups__0Image0.jpg
ten_of_cups__0Image0.jpg
page_of_cups__0Image0.jpg
knight_of_cups__0Image0.jpg
queen_of_cups__0Image0.jpg
king_of_cups__0Image0.jpg
the_suit

In [27]:
results = collection.query(
    query_texts=["What about relationtips"], # Chroma will embed this for you
    n_results=2  # how many results to return
)
print(results)

{'ids': [['id7', 'id14']], 'embeddings': None, 'documents': [[' As a potential partner, he is passionate and communicative', ' As an influence:  If two or more Kings fall close together in a reading, the meanings are as follows:   Two Kings:  A good partnership   Three Kings:  Influential men   Four Kings:  A power battle ']], 'uris': None, 'data': None, 'metadatas': [[None, None]], 'distances': [[1.4485702514648438, 1.6274409294128418]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


### Splitter из langchain

In [28]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=100,
    length_function=len,
)
split_documents = splitter.create_documents(docs)
len(split_documents)

223

In [29]:
str_split_documents = docs.copy()
for sentence in cards_splitted:
  str_split_documents = str_split_documents + [sentence[0]]

In [30]:
full_text = '\n'.join(str_split_documents)

In [31]:
splitter = CharacterTextSplitter(
    separator="\n",  # символ-разделитель, по умолчанию переход к новому абзацу '\n\n'
    chunk_size=300,  # размер документа в символах
    chunk_overlap=10,  # насколько соседние документы могут перекрывать друг-друга
    length_function=len,  # функция, по которой считается размер документа
    is_separator_regex=False,  # является ли разделитель регулярным выражением
)
split_documents = splitter.create_documents([full_text])
str_split_documents = [split.page_content for split in split_documents]
len(split_documents)

85

In [32]:
with open("/content/drive/MyDrive/taro_bot/full_text.txt", "w") as fp:
  fp.write(full_text)

## Эмбеддинги от HuggingFace

In [33]:
# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/LaBSE", model_kwargs={"device": "cpu"}
)

<ipython-input-33-33cee1d2bef9>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [34]:
embeddings = hf_embeddings_model.embed_documents(
    str_split_documents
)

In [35]:
with open('/content/drive/MyDrive/taro_bot/hf_embeddings_model.pkl', 'wb') as f:
    pickle.dump(hf_embeddings_model, f)

## Сохраняем Vector Store

In [36]:
db = FAISS.from_documents(
    split_documents, hf_embeddings_model
)

db.save_local("/content/drive/MyDrive/taro_bot/faiss_db")  # можно сохранить базу локально, указав путь

In [37]:
# Load the model from the file using pickle
with open('/content/drive/MyDrive/taro_bot/hf_embeddings_model.pkl', 'rb') as f:
    loaded_embeddings_model = pickle.load(f)

In [38]:
new_vector_store = FAISS.load_local(
    "/content/drive/MyDrive/taro_bot/faiss_db", loaded_embeddings_model, allow_dangerous_deserialization=True
)

In [39]:
# База готова, теперь можно делать к ней запросы
new_vector_store.similarity_search("What cards are arcana cards?")

[Document(id='426dc5d3-fe5e-40bf-96a0-4fa6122d50d4', metadata={}, page_content='The Minor Arcana Cards The remaining fifty-six cards are known collectively as the minor arcana, and they are arranged into four suits: Wands, Pentacles, Cups, and Swords. Each suit has fourteen cards, from Ace to Ten, plus four court cards: Page, Knight, Queen, and King. The four suits each have a ruling element, and each corresponds to specific areas of life:   Cups:  the element of Water; emotions and relationships'),
 Document(id='815d3c4a-49bd-4dc2-b9aa-978b244bb9d0', metadata={}, page_content='WHAT ABOUT REVERSALS? You will see that all the card interpretations in this book contain both upright and reversed card meanings. An upright card is just that—when you turn it over, it is upright. A card is known as reversed when it is upside down. There are lots of books dealing solely with reversed meanings. With a few exceptions, a reversed card’s meaning is generally more negative than the positive, or upri

## Определим Retriever

In [40]:
retriever = new_vector_store.as_retriever(
    search_type="mmr",  # тип поиска похожих документов
    k=1,  # количество возвращаемых документов (Default: 4)
    score_threshold=0.9,  # минимальный порог для поиска "similarity_score_threshold"
)

In [41]:
retriver_answer = retriever.get_relevant_documents(
    "What cards are arcana cards?"
)

<ipython-input-41-f6636567b36d>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriver_answer = retriever.get_relevant_documents(


In [42]:
retriver_str_answer = [split.page_content for split in retriver_answer]

In [43]:
retriver_str_answer[0]

'The Minor Arcana Cards The remaining fifty-six cards are known collectively as the minor arcana, and they are arranged into four suits: Wands, Pentacles, Cups, and Swords. Each suit has fourteen cards, from Ace to Ten, plus four court cards: Page, Knight, Queen, and King. The four suits each have a ruling element, and each corresponds to specific areas of life:   Cups:  the element of Water; emotions and relationships'

In [44]:
retriver_str_answer[1]

'A Beginner’s Guide to the Cards, Spreads, and Revealing the Mystery of the Tarot LIZ DEAN author of  The Art of Tarot  and  The Golden Tarot'

## Скачаем квантизованную Mistral

In [45]:
!pip install hydra-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.3 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=8c51d085afda4503720d416b6380827109959f41b364e47e35ba9f13c2d3a184
  Stored in directory: /root/.cache/pip/wheels/1a/97/32/461f837398029ad76911109f07047fde1d7b661a147c7c56d1
Successfully built antlr4-python3-runtime


In [46]:
import os

import hydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from hydra.core.global_hydra import GlobalHydra
from hydra.core.config_store import ConfigStore
from omegaconf import OmegaConf

In [48]:
with initialize_config_dir(version_base=None,
                           config_dir="/content/drive/MyDrive/hydra_configs/fine_tune",
                           job_name="fine_tune"):
    config = compose(config_name='config.yaml')

In [49]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 12.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=5959635 sha256=26b5027effaae6f5404cdce2530c09a896139693dfa27604773c5a27aa138b50
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python


In [52]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id='SleMary/my-new-repo',
                  local_dir="mistral-quantized",
                  local_dir_use_symlinks=False,
                  revision="main")

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Mistral-7B-Instruct-v0.3-Q8_0.gguf:   0%|          | 0.00/7.70G [00:00<?, ?B/s]

'/content/mistral-quantized'

In [53]:
import requests

API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"

from llama_cpp import Llama


def get_query(model_path):
    llm = Llama(model_path=model_path)

    def query(payload):
        output = llm(
            f"<USER>: {payload} <ASSISTANT>: ",
            max_tokens=128,
            stop=["Q:", "\n"],
        )
        return output
    return query


model_path = "/content/mistral-quantized/Mistral-7B-Instruct-v0.3-Q8_0.gguf"
query = get_query(model_path)
# =================================================

llama_model_loader: loaded meta data with 30 key-value pairs and 291 tensors from /content/mistral-quantized/Mistral-7B-Instruct-v0.3-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Mistral Norobots
llama_model_loader: - kv   3:                         general.size_label str              = 7.2B
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                       llama.context_length u32              = 32768
llama_model_loader: - kv   6:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   7:          

## Модель на cpu отвечает очеь долго, поэтому в таком случае лучше использовать api.

In [ ]:
output = query("What the answer to the universe is?")

print(output)

llama_perf_context_print:        load time =   37638.39 ms
llama_perf_context_print: prompt eval time =   37638.13 ms /    18 tokens ( 2091.01 ms per token,     0.48 tokens per second)
llama_perf_context_print:        eval time = 2620835.00 ms /    67 runs   (39116.94 ms per token,     0.03 tokens per second)
llama_perf_context_print:       total time = 2658826.29 ms /    85 tokens


{'id': 'cmpl-69acf7b2-64b5-41ce-94e5-cf9b1c8ba62f', 'object': 'text_completion', 'created': 1742933331, 'model': '/content/mistral-quantized/Mistral-7B-Instruct-v0.3-Q8_0.gguf', 'choices': [{'text': '42 is the answer to the ultimate question of life, the universe, and everything according to Douglas Adams\' science fiction series "The Hitchhiker\'s Guide to the Galaxy." However, the question itself is still up for interpretation. In reality, we may never truly know the answer to the universe\'s ultimate question.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 18, 'completion_tokens': 67, 'total_tokens': 85}}


In [ ]:
output = query("""
    Find a peaceful, relaxed feeling. Feel comfortable and confident.
    Imagine that you can hear the universe and are a good fortune teller.
    Question {}
    """.format("What weather will be tomorrow?")
)

print(output['choices'][0]['text'])

Llama.generate: 4 prefix-match hit, remaining 49 prompt tokens to eval
llama_perf_context_print:        load time =   37638.39 ms
llama_perf_context_print: prompt eval time =   48991.71 ms /    49 tokens (  999.83 ms per token,     1.00 tokens per second)
llama_perf_context_print:        eval time = 1675379.75 ms /    33 runs   (50769.08 ms per token,     0.02 tokens per second)
llama_perf_context_print:       total time = 1724565.75 ms /    82 tokens


 To find out the weather for tomorrow, I can check a reliable weather forecast website for you. Please give me the location for which you want to know the weather.


In [ ]:
output = query(
    """
    Imagine that you know everything about taro.
    Question: {}
    """.format("What cards are arcana cards?"),
)

print(output['choices'][0]['text'])

In [ ]:
output = query(
    """
    Imagine that you know everything about taro. Return only the detailed answer.
    Use the following information if it is useful:
    {}
    Question: {}
    """.format(retriver_str_answer[0], "What cards are arcana cards?"),
)

print(output['choices'][0]['text'])

# Основные функции

In [ ]:
# =================================================
def getter_query_function():
    llm = Llama(model_path=model_path)

    def get_query(additional_prompt, main_input):
        """
        Function that returns response from hugging face API open model.
        """

        # Combine the additional prompt and main input with a special token
        query = f"{additional_prompt} [END_PROMPT] {main_input}"

        output = llm(
            f"<USER>: {query} <ASSISTANT>: ",
            max_tokens=128,
            stop=["Q:", "\n"],
        )

        return output['choices'][0]['text']
    return get_query


get_query = getter_query_function()
# =================================================

In [ ]:
def get_retriever(retriever_answer, question='What can you tell about taro?'):
  """
  Функция забирает суммаризацию на предсказание одной карты.
  Здесь лучше всего работает mistral.
  """

  additional_prompt = """
      Imagine that you know everything about taro. Return only the detailed answer.
      Use the following information if it is useful:
      {}
      """.format(' '.join(retriever_answer))
  main_input = "{}".format(question)

  return get_query(additional_prompt, main_input)

In [ ]:
def get_one_card_prediction(random_card, topic=''):
  """
  Функция забирает суммаризацию на предсказание одной карты.
  Здесь лучше всего работает mistral.
  """

  additional_prompt = """
      Find a peaceful, relaxed feeling. Feel comfortable and confident.
      Imagine that you can hear the universe and are a good fortune teller.
      Make only three sentences.
      Try to use information:
      {}
      """.format(' '.join(random_card))
  main_input = "What is your Summirization? {}".format(topic)

  return get_query(additional_prompt, main_input)

In [ ]:
def get_yes_or_no(test_cards):
  """
  Функция  отвечает на вопрос Да или Нет.
  """

  additional_prompt = """
      If you get three yeses, your answer is certain; two yes cards
      (with one no or neutral) mean a positive outcome is most likely, but it may take
      time to come about; and all no or a mix of no/neutral cards means the answer to
      your question is, of course, negative.

      Yes cards: All cards apart from those listed here as no, neutral, or exceptions.

      No cards
      Swords: Three, Five, Six, Seven, Eight, Nine, Ten, and Knight
      Cups: Five, Seven, and Eight
      Pentacles: Five
      Death
      The Devil
      The Tower
      The Moon

      Neutral cards
      Swords: Four
      Cups: Four
      The Hermit
      The Hanged Man

      Exceptions
      Two of Swords or Ten of Wands: The answer is not yet known.
      Five and Seven of Wands: The answer is yes, but you must fight for your
      prize.
      Your cards are:
      {}
      """.format(' '.join(test_cards))
  main_input = "What is your answer, yes or no? And why?"

  return get_query(additional_prompt, main_input)

# Основные формы гаданий

In [ ]:
with open('/content/drive/MyDrive/taro_bot/cards_meaning_splits.pkl', 'rb') as f:
    cards_meaning_splits = pickle.load(f)

In [ ]:
the_taro_layout = "past, present, future"

if the_taro_layout == "question about taro":

  question = input()
  retriver_answer = retriever.get_relevant_documents(
    question
  )
  retriver_str_answer = [split.page_content for split in retriver_answer]
  answer = get_retriever(retriver_str_answer[0], question)

elif the_taro_layout == "one card":

  random_key = random.choice(list(cards_meaning_splits.keys()))
  additional_prompt = "One card for your events: "
  random_card = cards_meaning_splits[random_key]
  answer = ' '.join([additional_prompt, random_card])

elif the_taro_layout == "one card question":

  question = input()
  random_key = random.choice(list(cards_meaning_splits.keys()))
  random_card = cards_meaning_splits[random_key]
  answer = get_one_card_prediction(
      random_card,
      f' Answer on question {question}.'
      )

elif the_taro_layout == "past, present, future":

  times = {0: 'past', 1: 'present', 2: 'future'}
  answer = []
  for key, val in times.items():
    random_key = random.choice(list(cards_meaning_splits.keys()))
    random_card = cards_meaning_splits[random_key]
    card_answer = get_one_card_prediction(random_card, f' Answer in {val} tense.')
    answer.append( f"""{val} {card_answer}""")

  answer = ' '.join(answer)

elif the_taro_layout == "the celtic cross":

  questions = {
      0: "1 Your situation",
      1: "2 Responsibilities",
      2: "3 Limitations and the past",
      3: "4 What supports you",
      4: "5 What opposes you",
      5: "6 Achievements",
      6: "7 Attraction and relationships",
      7: "8 Work, health, and communication",
      8: "9 What is hidden",
      9: "10 The future environment; the outcome"
  }
  for key, val in questions.items():
    random_key = random.choice(list(cards_meaning_splits.keys()))
    random_card = cards_meaning_splits[random_key]

    answer = []
    card_answer = get_one_card_prediction(random_card, f' Answer on question {val}.')
    answer.append( f"""{val} {card_answer}""")

  ' '.join(answer)

elif the_taro_layout == "yes or no":

  test_cards = []
  for i in range(3):
    test_cards.append(random.choice(list(cards_meaning_splits.keys())))

  answer = get_yes_or_no(test_cards)

llama_perf_context_print:        load time =  150308.16 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   316 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   127 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  306734.38 ms /   443 tokens
Llama.generate: 58 prefix-match hit, remaining 339 prompt tokens to eval
llama_perf_context_print:        load time =  150308.16 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   339 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   110 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  270169.68 ms /   449 tokens
Llama.generate: 58 prefix-match hit, remaining 444 prompt tokens to eval
llama_perf_context_print:        load time =  150308.16 ms
llama_perf_context_print:

In [ ]:
answer

"past  Previously, the Four of Wands card indicated a period of success and celebration. The card's interpretation suggested that socially, one would have the opportunity to enjoy themselves, filled with confidence and vitality. This was often associated with the completion of a project, such as a building or remodeling, or a move to a larger property. The card's symbolism of putting down roots was reflected in establishing oneself in the community, becoming a pillar, and participating in local issues and social events. At work, the card signified a time when one's talent was appreciated, and they were full of ideas present  In the present, you are encouraged to evaluate your past decisions and actions. Pending great changes and opportunities, it is essential to address past issues that may need resolution, particularly regarding self-judgment and integrity. This process will allow you to acknowledge and praise yourself for your achievements. Additionally, Judgment suggests a spiritual

# Код бота
https://github.com/MaryIzo/taro_bot/tree/master